In [ ]:
#descargo de http://www.mindbigdata.com/opendb/

In [2]:
!pip install tensorflow

  Created wheel for opt-einsum: filename=opt_einsum-3.1.0-cp37-none-any.whl size=61701 sha256=fd2f7c9cec76420e4eeae8dc4a77c9ed560012ae01eaf61962cbd037a6753350
  Stored in directory: C:\Users\MATIAS\AppData\Local\pip\Cache\wheels\2c\b1\94\43d03e130b929aae7ba3f8d15cbd7bc0d1cb5bb38a5c721833
  Created wheel for absl-py: filename=absl_py-0.8.0-cp37-none-any.whl size=120992 sha256=74e903d73a1c70b62988fa6d77f19d12de44ad9fa4634b5465b79509f8a9e0d9
  Stored in directory: C:\Users\MATIAS\AppData\Local\pip\Cache\wheels\9a\1e\7a\456008eb5e47fd5de792c6139df6d5b3d5f71d51c6a0b94799
  Created wheel for termcolor: filename=termcolor-1.1.0-cp37-none-any.whl size=4835 sha256=c26b6a67b68447b10743213ce9d254f8737ca345678414dd5d47373dd934fdc2
  Stored in directory: C:\Users\MATIAS\AppData\Local\pip\Cache\wheels\7c\06\54\bc84598ba1daf8f970247f550b175aaaee85f68b4b0c5ab2c6
  Created wheel for gast: filename=gast-0.2.2-cp37-none-any.whl size=7546 sha256=55f7a3d3343ea71a1dde418496e4ea38e5db0afcdd2eb8d565801668ce76

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\ProgramData\Anaconda3\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "C:\ProgramData\Anaconda3\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: Error en una rutina de inicialización de biblioteca de vínculos dinámicos (DLL).

During handling of the above exception, another exceptio

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow_core\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\ProgramData\Anaconda3\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "C:\ProgramData\Anaconda3\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: Error en una rutina de inicialización de biblioteca de vínculos dinámicos (DLL).

During handling of the above exception, another exceptio

TypeError: can only concatenate str (not "list") to str

In [ ]:
def get_dataset(mnist_path = 'EP1.01.txt'):
    channels = ["AF3", "F7", "F3", "FC5", "T7", "P7", "O1", "O2", "P8", "T8", "FC6", "F4", "F8", "AF4"]
    X_tot, y_tot = [], []
    
    print('Reading file...')
    with open(mnist_path, 'r') as inp:
        print('Parsing data...')
        X, e, c = [], 0, 0
        for chunk in inp.readlines():
            try:
                id, event, device, channel, code, size, data = chunk.split('\t')
            except:
                e += 1
                print(e)
                pass
            signal = [float(val) for val in data.split(',')]
            X.append(signal)
            c += 1
            if c == 14:
                X = pad_sequences(X, maxlen = 256, dtype='float32')
                X_tot.append(X.tolist())
                y_tot.append(code)
                X, c = [], 0
        
       
    print('Done.')

    return X_tot, y_tot

In [ ]:
%%time
X_, y_ = get_dataset()

In [ ]:
X_ = np.array(X_)
y_ = np.array(y_, dtype='int8')
X_.shape, y_.shape

In [ ]:
randomize = np.arange(len(X_))
np.random.shuffle(randomize)
X = X_[randomize]
y = y_[randomize]

In [ ]:
classes = 10
d0 = 128   # Cantidad de neuronas en la primera capa de la red
d1 = 64   # Cantidad de neuronas de las "hiden layers"
batch = 250            # Tamanio de los batch en los que ingresan los mfcc a la red
main_epochs = 400           # Cantidad de veces que se entrenara la red con todos los mfcc
dropout = 0.3               # Dropout vertical
recurrent_dropout = 0.3
timesteps = 256

In [ ]:
# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

In [ ]:
# Parameters
learning_rate = 0.01
training_epochs = 20
batch_size = 256
display_step = 1
examples_to_show = 10

# Network Parameters
n_hidden_1 = 256 # 1st layer num features
n_hidden_2 = 128 # 2nd layer num features
n_input = 784 # MNIST data input (img shape: 28*28)

# tf Graph input (only pictures)
X = tf.placeholder("float", [None, n_input])

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([n_hidden_2, n_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([n_hidden_1, n_input])),
}
biases = {
    'encoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'encoder_b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'decoder_b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'decoder_b2': tf.Variable(tf.random_normal([n_input])),
}


# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['encoder_h1']),
                                   biases['encoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['encoder_h2']),
                                   biases['encoder_b2']))
    return layer_2


# Building the decoder
def decoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.add(tf.matmul(x, weights['decoder_h1']),
                                   biases['decoder_b1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.add(tf.matmul(layer_1, weights['decoder_h2']),
                                   biases['decoder_b2']))
    return layer_2

# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)

# Prediction
y_pred = decoder_op
# Targets (Labels) are the input data.
y_true = X

# Define loss and optimizer, minimize the squared error
cost = tf.reduce_mean(tf.pow(y_true - y_pred, 2))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(cost)

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    total_batch = int(mnist.train.num_examples/batch_size)
    # Training cycle
    for epoch in range(training_epochs):
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            _, c = sess.run([optimizer, cost], feed_dict={X: batch_xs})
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1),
                  "cost=", "{:.9f}".format(c))

    print("Optimization Finished!")

    # Applying encode and decode over test set
    encode_decode = sess.run(
        y_pred, feed_dict={X: mnist.test.images[:examples_to_show]})
    # Compare original images with their reconstructions
    f, a = plt.subplots(2, 10, figsize=(10, 2))
    for i in range(examples_to_show):
        a[0][i].imshow(np.reshape(mnist.test.images[i], (28, 28)))
        a[1][i].imshow(np.reshape(encode_decode[i], (28, 28)))
    f.show()
    plt.draw()
    plt.waitforbuttonpress()

In [ ]:
X = tf.placeholder('float64', shape = (None, 784), name = 'input')
y = tf.placeholder('int8', shape = (None, classes), name = 'output')

inp_shape = 764
n0 = 256
A = tf.Variable(tf.random.normal([n0, inp_shape]))
b = tf.Variable()

dense1 = tf.sigmoid(tf.matmul(X, A) + b)
dense2 = tf.softmax(tf.matmul(dense1, classes))

d1 = tf.layers.dense(X, d0, activation='relu')
d2 = tf.nn.softmax()

# NN

In [ ]:
from keras.layers import Dense, LSTM, CuDNNLSTM, GRU, BatchNormalization, TimeDistributed, UpSampling2D, MaxPooling2D, RepeatVector, AveragePooling1D, UpSampling1D, Reshape
from keras.layers import AveragePooling2D, Conv2D, Conv1D, Input, Flatten, MaxPool2D, ConvLSTM2D,  MaxPool1D, SeparableConv1D, SeparableConv2D, DepthwiseConv2D, Deconv2D
from keras import Model, Sequential

# Binario

In [ ]:
y_bin = y.copy()
y_bin[y != -1] = 0
y_bin[y == -1] = 1

In [ ]:
y[y == 1].shape

In [ ]:
X = X_.reshape([65034, timesteps, channels])

In [ ]:
inp = Input(shape = (timesteps, channels))

x = Conv1D(256, 16, activation='relu', input_shape=(timesteps, channels))(inp)
x = MaxPool1D(pool_size=6)(x)

x = Conv1D(128, 3, activation='relu')(x)
x = MaxPool1D(pool_size=4)(x)

x = SeparableConv1D(64, 3, activation='relu')(x)
x = MaxPool1D(pool_size=3)(x)

x = TimeDistributed(Flatten())(x)

x = CuDNNLSTM(input_neurons, return_sequences=True)(x)
x = CuDNNLSTM(hiden_neurons)(x)

x = Dense(28, activation = 'relu')(x)
out = Dense(1, activation='sigmoid')(x) 
model = Model(inputs = inp, outputs = out)

model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
model.summary()

In [ ]:
#model.fit(X, y, epochs = 20, batch_size = 256, validation_split=0.2)

# Clasificador Conv2d + LSTM

In [ ]:
y = to_categorical(y_, num_classes=n_classes)
X = X_.reshape([65034, timesteps, channels])

In [ ]:
X = X_.reshape([65034, channels, timesteps, 1])

# Clasificador Conv1D + LSTM

In [ ]:
X = X_.reshape([65034, timesteps, channels])

In [ ]:
#model.fit(X, y, epochs = 10, batch_size = 128)

In [ ]:
X.shape

In [ ]:
#para pasar de conv2d a 1d sacar una dimensino, y reshape x con menos 1 dim

In [ ]:
X_.shape

In [ ]:
n = 1
q = np.zeros(11)
q[n] = 1 
y[y == q].shape

In [ ]:
y.shape

# Autoencoders

In [ ]:
timesteps = 256
channels = 14

In [ ]:
#X = np.repeat(X_, 2, axis = 1)
X = X.reshape([65034, timesteps, channels, 1])
X.shape

In [ ]:
def autoencoder2d(input_img):
    #encoder
    #input = 28 x 28 x 1 (wide and thin)
    x = Conv2D(128, (16, 1), activation='relu', padding='same')(input_img)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = Conv2D(128, (16, 1), activation='relu', padding='same')(x)
    
    x = Conv2D(256, (1, 1), activation='linear')(x)
    
    #decoded = Conv2D(1, (1, 1), activation='sigmoid', padding='same')(x)
    
    return x

In [ ]:
inp = Input(shape=(timesteps, channels, 1))

autoencoder = Model(inp, autoencoder2d(inp))
autoencoder.summary()
autoencoder.compile(loss='mse', optimizer = 'adam')
#autoencoder.fit(X, X, epochs = 10, batch_size= 128)

In [ ]:
autoencoder.fit(X, X, epochs = 10, batch_size= 128)

In [ ]:
def autoencoder1d(input_img):
    #encoder
    #input = 28 x 28 x 1 (wide and thin)
    x = Conv1D(256, 32, activation='relu', padding='same')(input_img)
    x = MaxPool1D(pool_size=4)(x)
    x = Conv1D(128, 4, activation='relu', padding='same')(x)
    x = MaxPool1D(pool_size=4)(x)
    x = Conv1D(128, 4, activation='relu', padding='same')(x)
    x = MaxPool1D(pool_size=4)(x)
        
    x = UpSampling1D(4)(x)
    x = Conv1D(128, 4, activation='relu', padding='same')(x)
    x = UpSampling1D(4)(x)
    x = Conv1D(128, 4, activation='relu', padding='same')(x)
    x = UpSampling1D(4)(x)
    x = Conv1D(256, 32, activation='relu', padding='same')(x)
    x = Dense(28, activation = 'relu')(x)
    decoded = Dense(1, activation='sigmoid')(x)
    
    return decoded

In [ ]:
def autoencoderdense(input_img):
    #encoder
    #input = 28 x 28 x 1 (wide and thin)
    
    x = Dense(1024, activation='relu')(input_img)    
    x = Dense(512, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    
    
    x = Dense(64, activation='relu')(x)
    x = Dense(256, activation='relu')(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(1024, activation='relu')(x)    
    
    
    x = Dense(14, activation='relu')(x)
    
    
    return x

In [ ]:
#X = np.repeat(X_, 2, axis = 1)
X = X_.reshape([65034, timesteps * channels, 1])
X.shape

In [ ]:
#autoencoder.fit(X, X, epochs = 10, batch_size= 128)

In [ ]:
def autoencoder1d2(inp):
    
    x = Conv1D(512, 16, activation='relu', padding='same')(inp)
    x = Flatten()(x)
    x = Dense(512, activation = 'relu')(x) 

In [ ]:
X_.shape

In [ ]:
X = X_.reshape([65034, 14, 256])

In [ ]:
inp = Input(shape=(timesteps, channels, ))

autoencoder = Model(inp, autoencoderdense(inp))
autoencoder.summary()
autoencoder.compile(loss='mse', optimizer = 'adam')

# EEGnet


In [ ]:
from EEGModels import EEGNet

In [ ]:
X = X.reshape([65034, 1, 14, 256])
y_bin = np.zeros_like(y)
y_bin[y==-1] = 0
y_bin[y!=-1] = 1
y_bin = to_categorical(y_bin)
y_hot = to_categorical(y, num_classes=11)

In [ ]:
X.shape, y_bin.shape, y_hot.shape

In [ ]:
eegnet = EEGNet(11, Chans= 14, Samples = 256, F1=8, D=4, F2=16)
eegnet.summary()
eegnet.compile(loss="categorical_crossentropy", optimizer = 'adam', metrics=["accuracy"])
#eegnet.fit(X, y_hot, batch_size = 256, validation_split=0.1, epochs = 100)

In [ ]:
#eegnet.optimizer.lr = 0.0001
#eegnet.fit(X, y_hot, batch_size = 256, validation_split=0.15, epochs = 50)

In [ ]:
history = eegnet.history

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def plot_history(history):
    fig, ax = plt.subplots(12)
    # summarize history for accuracy
    ax[0].plot(history.history['acc'])
    ax[0].plot(history.history['val_acc'])
    ax[0].title('model accuracy')
    ax[0].ylabel('accuracy')
    ax[0].xlabel('epoch')
    ax[0].legend(['train', 'test'], loc='upper left')
    
    # summarize history for loss
    ax[1].plot(history.history['loss'])
    ax[1].plot(history.history['val_loss'])
    ax[1].title('model loss')
    ax[1].ylabel('loss')
    ax[1].xlabel('epoch')
    ax[1].legend(['train', 'test'], loc='upper left')

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()